# Desafio #QuarentenaDados

Bem-vinda e bem-vindo ao desafio #QuarentenaDados valendo um **Nintendo Switch**!

Esse notebook traz informações dos dados e como você deve configurar seu arquivo final para submissão.

**Caso queira usar esse notebook como exemplo para desenvolver seu projeto, clique em file e escolha a opção Save a copy in Drive**


Vamos trabalhar com uma amostra aleatória da base de dados MICRODADOS ENEM 2018, essa amostra é **diferente da apresentada em aula**. Junto com a divulgação do resultado final estaremos disponibilizando o código que gerou os dados para que você possa analisar e reproduzir os datasets. 

Seu objetivo é prever da melhor forma possível a nota das provas de **linguagens e códigos** (NU_NOTA_LC), dado todas as outras notas. O modelo que tiver o menor **erro quadrático médio (MSE)** vence o desafio.

Para o desafio você tem três bases à disposição, duas para desenvolver seu modelo e uma para submissão da predição. As bases são as seguintes:

- **dados_treino**: São 1500000 linhas contendo a nota das 4 provas + nota de redação.

- **dados_teste**: São 20000 linhas contendo com notas das 4 provas + nota de redação.

- **dados_desafioqt**: São 10000 linhas com nota de 3 provas + nota de redação. A nota da prova de **Linguagem e Codigos** (NU_NOTA_LC) não está disponível nessa base.


As base **dados_treino e dados_teste**, contém as seguintes colunas; **NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_MT, NU_NOTA_REDACAO, NU_NOTA_LC** (Você pode consultar a aula 5, onde o Guilherme explica o significado das siglas). A coluna que você deve realizar a **previsão** é **NU_NOTA_LC**. Você pode manipular os dados da forma que quiser, o importante é que no final submeta o arquivo com as informações corretas (detalhes da submissão serão discutidos no final deste notebook).

A base **dados_desafioqt**, contém as seguintes colunas; **ID, NU_NOTA_CN, NU_NOTA_CH, NU_NOTA_MT, NU_NOTA_REDACAO**. Repare que os dados **NU_NOTA_LC** não estão presentes, essa é justamente a informação que você precisa prever. Nós temos os valores reais das notas, no final do prazo de submissão um script irá avaliar sua previsão e dará uma nota para o seu modelo. Nessa base também temos o **ID**, essa informação é importante para o envio da sua previsão, garanta que a nota prevista corresponda ao respectivo **ID**.

Se você está habituado com o desenvolvimento de modelos de ML, repare que essa divisão de dados é exatamente a mesma que Treino, Teste e Validação. 

Abaixo preparamos um código exemplo para você seguir, sinta-se à vontade para experimentar diversos outros métodos, mas **GARANTA QUE O ARQUIVO DE SUBMISSÃO ESTEJA CONFIGURADO CORRETAMENTE**.

Na primeira parte, estamos lendo a base de dados direto de arquivos no github.






In [0]:
import pandas as pd

URI_TREINO = 'https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true'
URI_TESTE = 'https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv'
URI_DESAFIOQT = 'https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv'

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = 'Erro ao carregar dados de treino'
erro_teste = 'Erro ao carregar dados de teste'
erro_desafioqt = 'Erro ao carregar dados de submissão'

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

Agora com as bases de dados lidas, vamos separar as informações de cada dataset. X_treino e Y_treino são as **features**, X_teste e Y_teste são as **labels** a serem previstas.

Duas observações nesta parte:

- 1° Como já disponibilizamos os dados de treino e teste separados, você não precisa fazer *train_test_split* feito em aula (porém fique à vontade para trabalhar da forma que achar melhor).

- 2° Transformamos X_treino, Y_treino, X_teste, Y_teste em arrays numpy. Se você quiser usar uma biblioteca que não aceite dataframe como entrada de dados, já deixamos pronto para você.

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

A seguir criamos um modelo **Dummy** como exemplo e realizamos a avaliação do modelo com o **mean_squared_error**. 

Você pode usar qualquer algoritmo ou biblioteca para criar seus modelos, mas garanta que fará a avaliação com o mean_squared_error, pois usaremos essa métrica para avaliar sua predição final.

In [3]:
# Exemplo de classificação com Dummy

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

modelo_dummy = DummyRegressor()
modelo_dummy.fit(X_treino, Y_treino)
dummy_predicoes = modelo_dummy.predict(X_teste)

avaliacao_dummy = mean_squared_error(Y_teste, dummy_predicoes)

print(f'Minha avaliação nos dados de teste foi de {avaliacao_dummy}')

Minha avaliação nos dados de teste foi de 5219.286870186777


Depois que você criou testou e validou seu modelo, chegou a hora de preparar seu arquivo para a submissão do resultado.

No código abaixo, estamos realizando a predição das notas de **linguagem e códigos** do dataset **dados_desafioqt**. Feita a previsão, criamos um dataframe novo para a submissão, primeiro crimos a coluna **ID** e adicionamos a coluna **NU_NOTA_LC** com suas respectivas previsões (repare que nosso modelo não alterou as ordens dos ID's, mas se você utilizar algum modelo que embaralhe essa ordem certifique de colocar a previsão correta para o ID correto).

Após isso, salvamos o dataframe com ´.to_csv()´ (**importante, passe o parâmetro index=False para `.to_csv()`, caso contrário nosso script não computará sua nota**) no arquivo **PREDICAO_DESAFIOQT.csv (você precisa submeter o arquivo com esse nome, caso contrário nosso script de avaliação não computará sua nota**)  e utilizamos o `files.download` para baixar o arquivo em sua máquina local.

Feito tudo isso você está quase pronto para finalizar e submeter seu resultado. Você já baixou os dados, treinou e validou seu modelo, salvou sua previsão **no padrão ideal para submissão** e já está com o modelo baixado em sua máquina. Entretanto, ainda falta um detalhe: no momento de preencher o **forms** você precisa enviar seu código. Caso esteja usando os notebooks do colab siga as seguintes instruções para o download:

- Clique em **File** na parte superior esquerda.
- Depois selecione a opção **Download .ipynb** (também aceitaremos o .py caso você prefira desenvolver seu projeto em um arquivo python).


Pronto agora é só submeter seu resultado e torcer para levar um **Nintendo Switch** para casa.

Boa sorte!

## XGBoost Regressor

Importando bibliotecas necessárias.

In [0]:
import numpy as np
np.random.seed(43267)

import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

Criando as matrizes DMatrix que a API do XGBoost trabalha.

In [0]:
dmatrix_treino = xgb.DMatrix(data=X_treino, label=Y_treino)
dmatrix_teste = xgb.DMatrix(X_teste, label=Y_teste)

Criando um dicionário de parâmetros a serem ajustados.

In [0]:
params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': .3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
}

Adicionando a métrica RMSE de avaliação do nosso modelo.

In [0]:
params['eval_metric'] = 'rmse'

*num_boost_round* representa o número máximo de rodadas de boosting.

In [0]:
num_boost_round = 999

Treinando nosso modelo.

In [0]:
xgb_model = xgb.train(
    params,
    dmatrix_treino,
    num_boost_round=num_boost_round,
    evals=[(dmatrix_teste, 'Test')],
    early_stopping_rounds=10
)

Imprimindo os resultados.

In [13]:
print('Best RMSE: {:.2f} with {} rounds'
    .format(
        xgb_model.best_score,
        xgb_model.best_iteration+1))

Best RMSE: 45.46 with 15 rounds


Vamos ajustar os hiperparâmetros para alcançar um resultado melhor utilizando cross-validation.

In [15]:
cv_results = xgb.cv(
    params,
    dmatrix_treino,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=10
)

cv_results

[23:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:05:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,374.579883,0.039393,374.599011,0.174663
1,264.259058,0.031469,264.296417,0.191165
2,187.802536,0.027206,187.874252,0.203621
3,135.329364,0.029112,135.456256,0.214596
4,99.983926,0.028918,100.177034,0.211951
5,76.896747,0.036624,77.176160,0.200934
6,62.509361,0.042897,62.883800,0.189333
7,54.046838,0.049388,54.518878,0.188261
8,49.349554,0.051795,49.910563,0.187981
9,46.850671,0.049320,47.479739,0.192041


In [16]:
cv_results['test-rmse-mean'].min()

45.0380846

Ajustando os parâmetros *max_depth* e *min_child_weight*.

In [0]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9, 12)
    for min_child_weight in range(5, 8)
]

In [20]:
min_rmse = float('Inf')
best_params = None

for max_depth, min_child_weight in gridsearch_params:

    print('CV with max_depth={}, min_child_weight={}'
            .format(
              max_depth,
              min_child_weight))
    
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    
    cv_results = xgb.cv(
        params,
        dmatrix_treino,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print('\tRMSE {} for {} rounds'.format(mean_rmse, boost_rounds))
    
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth, min_child_weight)

print('Best params: {}, {}, RMSE: {}'
        .format(best_params[0], 
                best_params[1], 
                min_rmse))

CV with max_depth=9, min_child_weight=5
[23:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	RMSE 45.2334142 for 16 rounds
CV with max_depth=9, min_child_weight=6
[23:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Atualizando os parâmetros *max_depth* *min_child_weight* com os melhores valores encontrados.

In [0]:
params['max_depth'] = 9
params['min_child_weight'] = 6

Ajustando os parâmetros *subsample* e *colsample_bytree*.

In [0]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7, 11)]
    for colsample in [i/10. for i in range(7, 11)]
]

In [25]:
min_rmse = float('Inf')
best_params = None

for subsample, colsample in reversed(gridsearch_params):

    print('CV with subsample={}, colsample={}'
            .format(
              subsample,
              colsample))
    
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample

    cv_results = xgb.cv(
        params,
        dmatrix_treino,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print('\tRMSE {} for {} rounds'.format(mean_rmse, boost_rounds))
    
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample, colsample)

print('Best params: {}, {}, RMSE: {}'
        .format(
          best_params[0],
          best_params[1],
          min_rmse))

CV with subsample=1.0, colsample=1.0
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	RMSE 45.1682184 for 17 rounds
CV with subsample=1.0, colsample=0.9
[23:38:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:38

Atualizando os parâmetros *subsample* e *colsample_bytree* com os melhores valores encontrados.

In [0]:
params['subsample'] = 1.0
params['colsample_bytree'] = 1.0

Ajustando o parâmetro *ETA*.

In [34]:
%time

min_rmse = float('Inf')
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:

  print('CV with eta={}'.format(eta))
  params['eta'] = eta

  %time cv_results = xgb.cv(params, dmatrix_treino, num_boost_round=num_boost_round, seed=42, nfold=5, metrics=['rmse'], early_stopping_rounds=10)
  
  mean_rmse = cv_results['test-rmse-mean'].min()
  boost_rounds = cv_results['test-rmse-mean'].argmin()
  print('\tRMSE {} for {} rounds\n'.format(mean_rmse, boost_rounds))

  if mean_rmse < min_rmse:
      min_rmse = mean_rmse
      best_params = eta

print('Best params: {}, RMSE: {}'
        .format(best_params, 
                min_rmse))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
CV with eta=0.3
[23:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 36.4 s, sys: 43.9 ms, total: 36.4 s
Wall time: 9.48 s
	RMSE 45.1682184 for 17 rounds

CV with eta=0.2
[23:48:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:23] WARNING: /workspace/src/objec

Atualizando o parâmetro *ETA* com o melhor valor encontrado.

In [0]:
params['eta'] = .05

Parâmetros com seus valores ajustados.

In [42]:
params

{'colsample_bytree': 1.0,
 'eta': 0.05,
 'eval_metric': 'rmse',
 'max_depth': 9,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 1.0}

Treinando novamente nosso modelo com os parâmetros ajustados.

In [43]:
xgb_model = xgb.train(
    params,
    dmatrix_treino,
    num_boost_round=num_boost_round,
    evals=[(dmatrix_teste, "Test")],
    early_stopping_rounds=10
)

[00:00:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-rmse:505.985
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:480.886
[2]	Test-rmse:457.051
[3]	Test-rmse:434.419
[4]	Test-rmse:412.927
[5]	Test-rmse:392.523
[6]	Test-rmse:373.147
[7]	Test-rmse:354.75
[8]	Test-rmse:337.297
[9]	Test-rmse:320.726
[10]	Test-rmse:305.002
[11]	Test-rmse:290.078
[12]	Test-rmse:275.918
[13]	Test-rmse:262.484
[14]	Test-rmse:249.74
[15]	Test-rmse:237.653
[16]	Test-rmse:226.194
[17]	Test-rmse:215.326
[18]	Test-rmse:205.029
[19]	Test-rmse:195.267
[20]	Test-rmse:186.018
[21]	Test-rmse:177.262
[22]	Test-rmse:168.969
[23]	Test-rmse:161.119
[24]	Test-rmse:153.691
[25]	Test-rmse:146.668
[26]	Test-rmse:140.029
[27]	Test-rmse:133.755
[28]	Test-rmse:127.83
[29]	Test-rmse:122.235
[30]	Test-rmse:116.96
[31]	Test-rmse:111.988
[32]	Test-rmse:107.301
[33]	Test-rmse:102.887
[34]	Test-rmse:98.7393
[35]	Test-rmse:94.8417
[3

In [45]:
print('Best RMSE: {:.2f} in {} rounds'
        .format(xgb_model.best_score, 
                xgb_model.best_iteration + 1))

Best RMSE: 45.35 in 116 rounds


Salvando o melhor modelo construído.

In [46]:
num_boost_round = xgb_model.best_iteration + 1

best_model = xgb.train(
    params,
    dmatrix_treino,
    num_boost_round=num_boost_round,
    evals=[(dmatrix_teste, 'Test')]
)

[00:00:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-rmse:505.985
[1]	Test-rmse:480.886
[2]	Test-rmse:457.051
[3]	Test-rmse:434.419
[4]	Test-rmse:412.927
[5]	Test-rmse:392.523
[6]	Test-rmse:373.147
[7]	Test-rmse:354.75
[8]	Test-rmse:337.297
[9]	Test-rmse:320.726
[10]	Test-rmse:305.002
[11]	Test-rmse:290.078
[12]	Test-rmse:275.918
[13]	Test-rmse:262.484
[14]	Test-rmse:249.74
[15]	Test-rmse:237.653
[16]	Test-rmse:226.194
[17]	Test-rmse:215.326
[18]	Test-rmse:205.029
[19]	Test-rmse:195.267
[20]	Test-rmse:186.018
[21]	Test-rmse:177.262
[22]	Test-rmse:168.969
[23]	Test-rmse:161.119
[24]	Test-rmse:153.691
[25]	Test-rmse:146.668
[26]	Test-rmse:140.029
[27]	Test-rmse:133.755
[28]	Test-rmse:127.83
[29]	Test-rmse:122.235
[30]	Test-rmse:116.96
[31]	Test-rmse:111.988
[32]	Test-rmse:107.301
[33]	Test-rmse:102.887
[34]	Test-rmse:98.7393
[35]	Test-rmse:94.8417
[36]	Test-rmse:91.1791
[37]	Test-rmse:87.7455
[38]	Test-rms

In [47]:
mean_squared_error(best_model.predict(dmatrix_teste), Y_teste)

2056.585386990685

In [0]:
best_model.save_model('xgboost_regressor_enem_2018.model')

## Submissão

In [0]:
# Atribuir ao MODELO o nome do seu melhor modelo
from google.colab import files

MODELO = best_model

X_desafioqt = dados_desafioqt[coluna_features].to_numpy()
predicao_desafioqt = MODELO.predict(xgb.DMatrix(X_desafioqt))


desafio_df = pd.DataFrame(dados_desafioqt.ID)
desafio_df[coluna_label] = predicao_desafioqt

# NÃO TROCAR O NOME DO ARQUIVO DE SAÍDA (PREDICAO_DESAFIO)
desafio_df.to_csv('PREDICAO_DESAFIOQT.csv', index=False) 
files.download('PREDICAO_DESAFIOQT.csv')

In [50]:
X_desafioqt

array([[496.1, 585.3, 495.8, 320. ],
       [558.5, 641.9, 515.9, 600. ],
       [493.1, 566.6, 512. , 520. ],
       ...,
       [508.5, 598. , 511.7, 440. ],
       [530.9, 637.4, 630.9, 520. ],
       [467.7, 426.6, 533.2, 820. ]])